This notebook combines three Ben 10 datasets, alien attributes, enemy relationships, and battle history into a single master file. It merges each battle record with detailed stats for both the alien and its opponent, includes enemy network info, and outputs a master CSV file.

In [47]:
import pandas as pd

# Load the data
aliens = pd.read_csv('datasets/ben10_aliens.csv')
enemies = pd.read_csv('datasets/ben10_enemies.csv')
battles = pd.read_csv('datasets/ben10_battles.csv')

In [48]:
print("aliens:", aliens.columns.tolist())
print("enemies:", enemies.columns.tolist())
print("battles:", battles.columns.tolist())

aliens: ['alien_id', 'alien_name', 'species', 'home_planet', 'strength_level', 'speed_level', 'intelligence']
enemies: ['enemy_id', 'alien_name', 'enemy_name']
battles: ['battle_id', 'alien_name', 'enemy_name', 'battle_date', 'winner']


In [49]:
# Clean up column names (trim whitespace)
aliens.columns = aliens.columns.str.strip()
enemies.columns = enemies.columns.str.strip()
battles.columns = battles.columns.str.strip()

In [50]:
# Merge alien stats (attacker)
battles_merged = battles.merge(
    aliens, left_on='alien_name', right_on='alien_name', how='left', suffixes=('', '_alien1')
)
battles_merged = battles_merged.rename(columns={
    'species': 'alien1_species',
    'home_planet': 'alien1_home_planet',
    'strength_level': 'alien1_strength_level',
    'speed_level': 'alien1_speed_level',
    'intelligence': 'alien1_intelligence'
})


In [51]:
# Merge enemy stats (defender)
battles_merged = battles_merged.merge(
    aliens, left_on='enemy_name', right_on='alien_name', how='left', suffixes=('', '_alien2')
)
battles_merged = battles_merged.rename(columns={
    'species': 'alien2_species',
    'home_planet': 'alien2_home_planet',
    'strength_level': 'alien2_strength_level',
    'speed_level': 'alien2_speed_level',
    'intelligence': 'alien2_intelligence'
})

In [52]:
# Merge enemy relationship for enemy_id
battles_merged = battles_merged.merge(
    enemies[['enemy_id', 'alien_name', 'enemy_name']],
    on=['alien_name', 'enemy_name'],
    how='left'
)

In [53]:
# Drop all id columns and duplicate 'alien_name' columns
cols_to_drop = [
    'alien_id', 'enemy_id', 'battle_id, enemy_id, battle_date',
]
for col in cols_to_drop:
    if col in battles_merged.columns:
        battles_merged = battles_merged.drop(columns=[col])

In [54]:
# Select and order final columns
final_columns = [
    'alien_name',
    'enemy_name',
    'alien1_species',
    'alien1_home_planet',
    'alien1_strength_level',
    'alien1_speed_level',
    'alien1_intelligence',
    'alien2_species',
    'alien2_home_planet',
    'alien2_strength_level',
    'alien2_speed_level',
    'alien2_intelligence',
    'winner'
]
battles_final = battles_merged[final_columns]

# Save to CSV
battles_final.to_csv('datasets/ben10_master.csv', index=False)
print("Master file created as ben10_master.csv")

Master file created as ben10_master.csv
